In [103]:
import pandas as pd
import os

os.chdir('/home/pier/Machine_Learning/fifa18_final16')
df_results = pd.read_csv('data/results.csv')
df_results = df_results[df_results['date'] > '2015-1-1']
df_results
# df_new = df_results[(df_results['home_team'].str.contains("Germany")) | (df_results['away_team'].str.contains("Germany"))]
# df_new


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
37119,2015-10-02,Oman,Syria,2,1,Friendly,Muscat,Oman,False
37120,2015-10-03,Jordan,Iraq,3,0,Friendly,Amman,Jordan,False
37121,2015-10-07,Comoros,Lesotho,0,0,FIFA World Cup qualification,Moroni,Comoros,False
37122,2015-10-07,Mauritius,Kenya,2,5,FIFA World Cup qualification,Mapou,Mauritius,False
37123,2015-10-07,Seychelles,Burundi,0,1,FIFA World Cup qualification,Roche Caiman,Seychelles,False
37124,2015-10-07,South Sudan,Mauritania,1,1,FIFA World Cup qualification,Juba,South Sudan,False
37125,2015-10-07,Tanzania,Malawi,2,0,FIFA World Cup qualification,Dar es Salaam,Tanzania,False
37126,2015-10-08,Albania,Serbia,0,2,UEFA Euro qualification,Elbasan,Albania,False
37127,2015-10-08,Argentina,Ecuador,0,2,FIFA World Cup qualification,Buenos Aires,Argentina,False
37128,2015-10-08,Bahrain,Uzbekistan,0,4,FIFA World Cup qualification,Riffa,Bahrain,False


In [104]:

os.chdir('/home/pier/Machine_Learning/fifa18_final16')
df_rankings = pd.read_csv('data/fifa_ranking.csv')
df_rankings = df_rankings[df_rankings['rank_date'].str.contains('2018')]
df_rankings =df_rankings.groupby('country_full')['rank', 'cur_year_avg_weighted'].mean()
df_rankings
# df_test = pd.DataFrame(df_test)
# df_test.reset_index()
df_rankings['country_full'] = df_rankings.index
# df_test[df_test['country_full'].str.contains('Germany')]
# df_test


# Join into the main dataset and cleaning

In [105]:
df_combined = df_results.set_index('home_team').join(df_rankings.set_index('country_full'))
df_combined.rename(columns={'rank' : 'home_rank', 'cur_year_avg_weighted' : 'home_weighted_points'}, inplace=True)
df_combined['home_team'] = df_combined.index
df_combined = df_combined.set_index('away_team').join(df_rankings.set_index('country_full'))
df_combined.rename(columns={'rank' : 'away_rank', 'cur_year_avg_weighted' : 'away_weighted_points'}, inplace=True)
df_combined['away_team'] = df_combined.index

In [111]:
df_combined = df_combined.dropna()
df_combined.drop(['date', 'city', 'country'], axis=1, inplace=True)


In [112]:
df_combined

,home_score,away_score,tournament,neutral,home_rank,home_weighted_points,home_team,away_rank,away_weighted_points,away_team
Afghanistan,0,3,SAFF Cup,True,185.833333,0.000000,Bhutan,145.166667,104.065000,Afghanistan
Afghanistan,1,0,AFC Asian Cup qualification,False,169.833333,42.690000,Cambodia,145.166667,104.065000,Afghanistan
Afghanistan,2,1,SAFF Cup,False,99.000000,202.945000,India,145.166667,104.065000,Afghanistan
Afghanistan,5,0,FIFA World Cup qualification,False,57.833333,220.966667,Japan,145.166667,104.065000,Afghanistan
Afghanistan,4,1,AFC Asian Cup qualification,False,115.166667,140.725000,Jordan,145.166667,104.065000,Afghanistan
Afghanistan,2,0,Friendly,False,82.666667,306.000000,Lebanon,145.166667,104.065000,Afghanistan
Afghanistan,1,1,Friendly,False,173.333333,25.450000,Malaysia,145.166667,104.065000,Afghanistan
Afghanistan,2,0,Friendly,False,93.833333,228.235000,Oman,145.166667,104.065000,Afghanistan
Afghanistan,1,0,FIFA World Cup qualification,False,171.666667,31.260000,Singapore,145.166667,104.065000,Afghanistan
Afghanistan,5,2,FIFA World Cup qualification,True,75.000000,232.328333,Syria,145.166667,104.065000,Afghanistan


# Feature generation

In [113]:
# Taking home team to be the one we are interested in 
df_combined['won'] = (df_combined['home_score'] - df_combined['away_score']) > 1

In [115]:
df_combined['rank_diff'] = df_combined['home_rank'] - df_combined['away_rank']

In [116]:
df_combined['point_diff'] = df_combined['home_weighted_points'] - df_combined['away_weighted_points']

In [118]:
df_combined.to_csv('combined.csv')